In [1]:
!pip install catboost

     |████████████████████████████████| 69.2MB 47kB/s 


Вам предоставлен перечень BigMedia промо-акций за последний квартал 2019-ого года и весь 2020-ый год, а также чековые данные для случайной выборки из клиентов "Ленты" за период с 20 сентября 2019 года по 20 сентября 2020 года и таблица с товарной иерархией. На основе этих данных, вам необходимо построить модель UpLift’а промо-кампаний и сделать предсказания для кампаний, проведённых в 4-ом квартале 2020-ого года


In [83]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from catboost import Pool
import os
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error  #for metric of our model

RS=42   #we will fix random state

In [84]:
# подключаем гугл диск на котором данные
from google.colab import drive
drive.mount ('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [85]:
# Путь к директории с данными
data_dir = '../content/gdrive/My Drive/dataset2021_promo/'
os.listdir(data_dir)

['20210518_checks.csv',
 '20210518_hierarchy.csv',
 '20210518_uplift.csv',
 '20210521_offers.csv',
 '20210521_sample_submission.csv']

In [86]:
uplift_df = pd.read_csv(data_dir+'20210518_uplift.csv')
uplift_df.tail(2)

,Offer_ID,UpLift
738,687,5.097561
739,689,3.949051


1.	uplift.csv - 2 столбца - Offer_ID и UpLift - для каждого оффера из трейна значение таргета, который нужно будет предсказывать. Ключ - Offer_ID

In [87]:
offers_df = pd.read_csv(data_dir+'20210521_offers.csv')
offers_df.tail(2)

,Promo_type,Offer_ID,sku,start_date,end_date,train_test_group
9774,Biweekly,690,40d78b3648a4ea15f300070a0ffd9752,20191114,20191127,train
9775,Biweekly,687,0d73ba0d16f74777b696657bf69a9ef0,20191114,20191127,train


In [88]:
offers_df.train_test_group.value_counts()

train    7716
test     2060
Name: train_test_group, dtype: int64

In [89]:
from datetime import datetime
date_New_Year = datetime.strptime('20210101', '%Y%m%d')
offers_df['Xmas_days'] = list(map(
    lambda x: (date_New_Year - datetime.strptime(str(x), '%Y%m%d')).days, offers_df['end_date'])
)
offers_df.head(2)


,Promo_type,Offer_ID,sku,start_date,end_date,train_test_group,Xmas_days
0,Facades,F26,71c770923102af052f64c6036c9dd590,20200213,20200219,train,317
1,Facades,F2,e87788b585ead792a4d40f7de18f2f76,20200102,20200108,train,359


In [90]:
offers_df['Xmas_days_start'] = list(map(
    lambda x: (date_New_Year - datetime.strptime(str(x), '%Y%m%d')).days, offers_df['start_date'])
)
offers_df['qty_days'] = offers_df['Xmas_days'] - offers_df['Xmas_days_start']
offers_df.head(2)

,Promo_type,Offer_ID,sku,start_date,end_date,train_test_group,Xmas_days,Xmas_days_start,qty_days
0,Facades,F26,71c770923102af052f64c6036c9dd590,20200213,20200219,train,317,323,-6
1,Facades,F2,e87788b585ead792a4d40f7de18f2f76,20200102,20200108,train,359,365,-6


In [91]:
hierarachy_df = pd.read_csv(data_dir+'20210518_hierarchy.csv')
hierarachy_df.tail(2)

,sku,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4
80404,15cafb345d5e95b3ae83f2cbb87e0f72,44a7b825383402ed937a15fb8a5bb12a,d2ddea18f00665ce8623e36bd4e3c7c5,a523426cc585745318d5f6d91a9c0706,846a6cd5242ad9c4e8eea0621a9cf252
80405,7bda33b7e9d73cd44599a25411fe9ade,cc9284597026dcc021735c0970ca0c93,6ea9ab1baa0efb9e19094440c317e21b,9e82757e9a1c12cb710ad680db11f6f1,a6ba2f3ff8fddfa4546aa9331601cfa1


In [92]:
offers_df = pd.merge(offers_df,hierarachy_df,on='sku')  #merge 
offers_df.tail(2)

,Promo_type,Offer_ID,sku,start_date,end_date,train_test_group,Xmas_days,Xmas_days_start,qty_days,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4
9774,Biweekly,690,e42fff243b2395bfb40640dc398d86ea,20191114,20191127,train,401,414,-13,44a7b825383402ed937a15fb8a5bb12a,f7177163c833dff4b38fc8d2872f1ec6,2120606d6a751a93e392b46d945bba9d,52cbb5dbb41a50faf776f1a19ba39402
9775,Biweekly,690,dcb69d7f01aef0327bd4887d4629f74f,20191114,20191127,train,401,414,-13,44a7b825383402ed937a15fb8a5bb12a,f7177163c833dff4b38fc8d2872f1ec6,2120606d6a751a93e392b46d945bba9d,52cbb5dbb41a50faf776f1a19ba39402


In [93]:
test_df = offers_df.loc[offers_df['train_test_group'] == 'test']  #test
test_df.tail(2)

,Promo_type,Offer_ID,sku,start_date,end_date,train_test_group,Xmas_days,Xmas_days_start,qty_days,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4
9363,Biweekly,535,c22f24d440d089f13edaac2d5a782be3,20201029,20201111,test,51,64,-13,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,c09f9caf5e08836d4673ccdd69bb041e,040d3d534731bc59248d236d654ee63a
9364,Seasonal,99,fc2fe174d3547a2d077bb94742977bdb,20201006,20201019,test,74,87,-13,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,3341f6f048384ec73a7ba2e77d2db48b,44ffc63b904a643b73198d4c72d95473


In [94]:
test_df.shape

(2060, 13)

In [95]:
train_df = pd.merge(offers_df,uplift_df,on='Offer_ID')  #merge 
train_df.tail(2)

,Promo_type,Offer_ID,sku,start_date,end_date,train_test_group,Xmas_days,Xmas_days_start,qty_days,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4,UpLift
7714,Seasonal,674,40984242750480edfbeed035b3e34dae,20191008,20191104,train,424,451,-27,44a7b825383402ed937a15fb8a5bb12a,7f39f8317fbdb1988ef4c628eba02591,56880339cfb8fe04c2d17c6160d0512f,836eeb4749fbccf1f711f887304e6e38,14.977547
7715,Seasonal,674,3a4641a5b75c395801c4c1d31bca9191,20191008,20191104,train,424,451,-27,44a7b825383402ed937a15fb8a5bb12a,7f39f8317fbdb1988ef4c628eba02591,56880339cfb8fe04c2d17c6160d0512f,836eeb4749fbccf1f711f887304e6e38,14.977547


In [96]:
# Разделим годы на трейн и тест
X_features = ['Promo_type','sku','hierarchy_level1','hierarchy_level2','hierarchy_level3','hierarchy_level4','Xmas_days','qty_days']
X_train, X_test, y_train, y_test = train_test_split(train_df[X_features], 
                                                    train_df['UpLift'], 
                                                    test_size=0.2,  
                                                    shuffle=True, 
                                                    random_state=RS)

In [97]:

target_col = 'UpLift'
text_cols = []
categorical_cols = ['Promo_type','sku','hierarchy_level1','hierarchy_level2','hierarchy_level3','hierarchy_level4']

train_pool = Pool(
        X_train, 
        y_train, 
        cat_features=categorical_cols,
        text_features=text_cols,
        feature_names=X_features,
    )
valid_pool = Pool(
        X_test, 
        y_test, 
        cat_features=categorical_cols,
        text_features=text_cols,
        feature_names=X_features,
    )


In [98]:
regr_CB =CatBoostRegressor(iterations= 4800,
    learning_rate= 0.01,
    depth=16,
    task_type= 'GPU',
    early_stopping_rounds= 90,
    #class_weights=[0.95,0.05],
    use_best_model= True,
    random_seed=RS,
    verbose= 60
)

In [99]:
regr_CB.fit(train_pool, eval_set=valid_pool,plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 4.9962881	test: 5.1284911	best: 5.1284911 (0)	total: 58.2ms	remaining: 4m 39s
60:	learn: 4.6799480	test: 4.7669654	best: 4.7669654 (60)	total: 4.67s	remaining: 6m 2s
120:	learn: 4.5185714	test: 4.5883040	best: 4.5883040 (120)	total: 9.13s	remaining: 5m 52s
180:	learn: 4.4243502	test: 4.4819311	best: 4.4819311 (180)	total: 13.8s	remaining: 5m 52s
240:	learn: 4.3625953	test: 4.4147741	best: 4.4147741 (240)	total: 19s	remaining: 5m 59s
300:	learn: 4.2975044	test: 4.3493406	best: 4.3493406 (300)	total: 23s	remaining: 5m 43s
360:	learn: 4.2376902	test: 4.2864365	best: 4.2864365 (360)	total: 29.8s	remaining: 6m 6s
420:	learn: 4.1833638	test: 4.2309388	best: 4.2309388 (420)	total: 34.3s	remaining: 5m 56s
480:	learn: 4.1357944	test: 4.1870707	best: 4.1870707 (480)	total: 38.2s	remaining: 5m 42s
540:	learn: 4.0883639	test: 4.1408153	best: 4.1408153 (540)	total: 45.1s	remaining: 5m 55s
600:	learn: 4.0250969	test: 4.0800386	best: 4.0800386 (600)	total: 50.2s	remaining: 5m 50s
660:	learn

In [100]:
#let s make prediction
prediction_val = regr_CB.predict(X_test)
print(prediction_val[:10])

[10.94728266  0.2432939   5.53224842  4.56388798  4.05384355  1.99240317
  2.27123508  2.40884595  2.67928998  2.85841511]


In [101]:
#evaluate our model
err_val = mean_absolute_error(prediction_val, y_test)
print(err_val)

2.060367940273257


In [102]:
regr_CB.feature_importances_

array([ 9.58554555,  2.8566205 ,  0.42801112, 26.86379186, 12.92457091,
       20.18453516, 23.24765111,  3.90927379])

In [103]:
print(X_features[0],X_features[1],X_features[2],X_features[3],X_features[4],X_features[5],X_features[6],X_features[7])

Promo_type sku hierarchy_level1 hierarchy_level2 hierarchy_level3 hierarchy_level4 Xmas_days qty_days


In [104]:
sample_df = pd.read_csv(data_dir+'20210521_sample_submission.csv')
sample_df.tail(2)

,Offer_ID,UpLift
147,95,0
148,97,0


In [105]:
test_df = pd.merge(sample_df,offers_df,on='Offer_ID')  #merge 
test_df.tail(2)

,Offer_ID,UpLift,Promo_type,sku,start_date,end_date,train_test_group,Xmas_days,Xmas_days_start,qty_days,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4
2058,97,0,Seasonal,fc2b9dd2eb73d0a98bb327b784a50e8c,20201006,20201019,test,74,87,-13,44a7b825383402ed937a15fb8a5bb12a,44f683a84163b3523afe57c2e008bc8c,62db9e3397c76207a687c360e0243317,41e84c7eeeb165f4797425ee7ff0bbb4
2059,97,0,Seasonal,d4a0e64e706662edf908da48ff560917,20201006,20201019,test,74,87,-13,44a7b825383402ed937a15fb8a5bb12a,44f683a84163b3523afe57c2e008bc8c,62db9e3397c76207a687c360e0243317,41e84c7eeeb165f4797425ee7ff0bbb4


In [106]:
pred_test = regr_CB.predict(test_df[X_features])
print(pred_test[:10])

[6.71957168 2.96032493 2.96032493 3.30339393 3.12599098 3.30339393
 3.12599098 2.96032493 3.30339393 2.96032493]


In [107]:
test_df['UpLift'] = pred_test
test_df.tail(2)

,Offer_ID,UpLift,Promo_type,sku,start_date,end_date,train_test_group,Xmas_days,Xmas_days_start,qty_days,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4
2058,97,3.868015,Seasonal,fc2b9dd2eb73d0a98bb327b784a50e8c,20201006,20201019,test,74,87,-13,44a7b825383402ed937a15fb8a5bb12a,44f683a84163b3523afe57c2e008bc8c,62db9e3397c76207a687c360e0243317,41e84c7eeeb165f4797425ee7ff0bbb4
2059,97,3.868015,Seasonal,d4a0e64e706662edf908da48ff560917,20201006,20201019,test,74,87,-13,44a7b825383402ed937a15fb8a5bb12a,44f683a84163b3523afe57c2e008bc8c,62db9e3397c76207a687c360e0243317,41e84c7eeeb165f4797425ee7ff0bbb4


In [108]:
sub_df = test_df.groupby('Offer_ID').agg({'UpLift':np.mean})
sub_df.tail(2)

,UpLift
Offer_ID,
F219,6.547104
F220,8.365905


In [109]:
sub_df.shape

(149, 1)

In [110]:
sub_dict = sub_df.to_dict()
print(sub_dict)

{'UpLift': {'100': 2.8504046673056913, '172': 6.166400236394493, '173': 5.252905541149622, '174': 2.9317852248474203, '175': 3.345716188295534, '176': 5.23969221897851, '177': 4.375101763659842, '178': 5.596792281230561, '181': 7.6022675497175705, '182': 3.1405842910735555, '228': 5.945086568318565, '229': 3.6845373747567454, '230': 4.230552387481513, '231': 5.141167265568738, '232': 6.131429689078423, '233': 4.972200573503203, '234': 3.047764626532573, '236': 3.538615066585062, '316': 1.0778259310033143, '317': 1.6723917675803948, '318': 2.386235746230905, '319': 1.5139609732066974, '320': 1.5328940900665213, '321': 2.4642434556287527, '322': 2.287235249628721, '370': 8.23596429020802, '372': 3.7491992294451304, '374': 6.713816382674057, '375': 2.720683246736575, '376': 10.819149877422738, '377': 12.707734882775984, '378': 1.608606324897144, '379': 5.067679082460812, '380': 7.2094009448626055, '381': 7.621428983080083, '382': 6.453918590445578, '383': 2.5505105487435045, '386': 6.7195

In [111]:
test_offer_id_list = sample_df['Offer_ID']
print(test_offer_id_list)

0      386
1      182
2      173
3      375
4      379
      ... 
144    536
145     99
146    100
147     95
148     97
Name: Offer_ID, Length: 149, dtype: object


In [112]:
lst = []
for i in test_offer_id_list:
  lst.append(sub_dict['UpLift'][i])
  #lst.append(sub_dict['UpLift'][i]-1.0)
print(lst)

[6.719571677532262, 3.1405842910735555, 5.252905541149622, 2.720683246736575, 5.067679082460812, 2.1804759527926088, 12.707734882775984, 7.621428983080083, 2.725782223437996, 5.596792281230561, 6.166400236394493, 10.819149877422738, 2.9317852248474203, 2.5505105487435045, 7.6022675497175705, 4.100718962390809, 6.453918590445578, 5.23969221897851, 4.375101763659842, 6.713816382674057, 1.608606324897144, 8.23596429020802, 3.345716188295534, 3.7491992294451304, 7.2094009448626055, 2.163160082905428, 0.9120872941272289, 6.10610205341391, 2.9521872991588425, 5.574380645182973, 0.9027056718062075, 0.6604858540677958, 0.6702313960579639, 0.5674569438505745, 2.4654625135868184, 2.5524909725132297, 8.863727181811537, 1.9314864046607945, 0.8362765998666495, 0.8609799394743227, 0.928785519575495, 1.566948839223188, 2.4509056716456605, 3.3669262798771613, 1.1851144059151024, 4.954675584057558, 1.4017999558941838, 4.111998753334824, 0.6325731576528538, 1.636252311178385, 1.1050845472637647, 3.25566

In [113]:
sample_df['UpLift'] = lst
sample_df.tail(2)

,Offer_ID,UpLift
147,95,2.673682
148,97,3.868015


In [114]:
#let s save our prediction

sample_df.to_csv('sub12.csv',header=True,index=False)

from google.colab import files

files.download('sub12.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>